In [1]:
import cv2
import tensorflow as tf
import numpy as np
import math
import time
import os
import pyttsx3 
from gtts import gTTS
def paa_skin(image,image1):  
    
    new1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
    cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
    return image1,dist
def callback(x):
    pass
def gesture(cX0,cX1,cY0,cY1,area_diff):
    x=abs(cX1-cX0)
    y=abs(cY1-cY0)
     
    
    if(x<150 and y<100):
        if area_diff>0:
            move="front"
        if area_diff<0:
            move="back"
        return move
    if(x>y and x>150):
        if(cX1>cX0 and area_diff>0):
            move="Left and front"
        elif(cX1>cX0 and area_diff<0):
            move="Left and back"
        elif(cX1<cX0 and area_diff<0):
            move="Right and back"
        else:
            move="Right and front"
    if(x<y and y>100):
        if(cY1>cY0 and area_diff>0):
            move="Down and front"
        elif(cY1>cY0 and area_diff<0):
            move="Down and back"
        elif(cY1<cY0 and area_diff>0):
            move="Up and front"
        else:
            move="Up and back"
    return move 

In [2]:
cv2.namedWindow('image1')
cap=cv2.VideoCapture(0)
ilowH = 0
ihighH = 255
ilowS = 0
ihighS = 255
ilowV = 0
ihighV = 255
cv2.createTrackbar('lowH1', 'image1', ilowH, 179, callback)
cv2.createTrackbar('highH1', 'image1', ihighH, 179, callback)
cv2.createTrackbar('lowS1', 'image1', ilowS, 255, callback)
cv2.createTrackbar('highS1', 'image1', ihighS, 255, callback)
cv2.createTrackbar('lowV1', 'image1', ilowV, 255, callback)
cv2.createTrackbar('highV1', 'image1', ihighV, 255, callback)
while True:
    try:
        ret,frame=cap.read()
        ilowH1 = cv2.getTrackbarPos('lowH1', 'image1')
        ihighH1 = cv2.getTrackbarPos('highH1', 'image1')
        ilowS1 = cv2.getTrackbarPos('lowS1', 'image1')
        ihighS1 = cv2.getTrackbarPos('highS1', 'image1')
        ilowV1 = cv2.getTrackbarPos('lowV1', 'image1')
        ihighV1 = cv2.getTrackbarPos('highV1', 'image1')
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        low1=np.array([ilowH1,ilowS1,ilowV1])
        high1=np.array([ihighH1,ihighS1,ihighV1])
        img_mask1=cv2.inRange(hsv,low1,high1)
        output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
        new1 = cv2.cvtColor(output1, cv2.COLOR_BGR2GRAY)
        dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
        cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
        M=cv2.moments(dist)
        cX = (int(M["m10"] / M["m00"]))
        cY = (int(M["m01"] / M["m00"]))
        cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)
        cv2.imshow("image1",frame)
        cv2.imshow('dist',dist)
        cv2.imshow("image",output1)
    except ZeroDivisionError:
        pass
    if (cv2.waitKey(1) == 13):
                break
cv2.destroyAllWindows()
cap.release()
i=0

In [ ]:
output1.size

In [15]:
cap=cv2.VideoCapture(0)
start=time.time()
black=np.zeros((output1.shape[0],output1.shape[1]))
j=1
while True:
    try:
        ret,frame=cap.read()
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        low1=np.array([ilowH1,ilowS1,ilowV1])
        high1=np.array([ihighH1,ihighS1,ihighV1])
        img_mask1=cv2.inRange(hsv,low1,high1)
        output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
        new1 = cv2.cvtColor(output1, cv2.COLOR_BGR2GRAY)
        dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
        cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
        M=cv2.moments(dist)
        cX = (int(M["m10"] / M["m00"]))
        cY = (int(M["m01"] / M["m00"]))
        
        contours, hierarchy = cv2.findContours(new1.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
        area = cv2.contourArea(sorted_contours[0])
        
        cv2.circle(frame, (cX, cY), 5, (255, 252, 255), -1)
        cv2.circle(black, (cX, cY), 5, (255, 255, 255), -1)
        if(j==1):
            cX0=cX
            cY0=cY
            initial_area=area
        if(j!=1):
            cv2.line(black,(sX,sY),(cX,cY),(255,255,255),5)
        cv2.imshow("image1",frame)
        cv2.imshow("image",output1)
        cv2.imshow("path",black)
        j=j+1
        sX=cX
        sY=cY
        end=time.time()
        diff=end-start
        if (diff>3):
            cX1=cX
            cY1=cY
            final_area=area
            #cv2.putText(black,"Start",(cX0,cY0),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,255,255),lineType=cv2.LINE_AA)
            #cv2.putText(black,"End",(cX1,cY1),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,255,255),lineType=cv2.LINE_AA)
            area_diff=final_area-initial_area
            q=gesture(cX0,cX1,cY0,cY1,area_diff)
            
            engine = pyttsx3.init()
            black=cv2.flip(black,1)
            cv2.imwrite("./waste/"+str(i)+".jpeg",black)
            hi=cv2.imread("./waste/"+str(i)+".jpeg")
            cv2.putText(hi,str(q),(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,255,255),lineType=cv2.LINE_AA)
            cv2.imshow("black",hi)
            cv2.imwrite("/waste/"+str(i)+".jpeg",black)
            black=np.zeros((output1.shape[0],output1.shape[1]))
            i=i+1
            j=1
            start=time.time()
            #print("hi")|
            print("xaxis "+str(cX1-cX0)+" yaxis "+str(cY1-cY0)+" area "+str(area_diff))
#             engine.say(q)
#             engine.runAndWait()        
    except ZeroDivisionError:
        pass
    if (cv2.waitKey(1) == 13):
                break
cv2.destroyAllWindows()
cap.release()

xaxis 29 yaxis -69 area -6019.5
xaxis 8 yaxis 44 area 5469.5
xaxis -224 yaxis -10 area 902.0
xaxis 365 yaxis -16 area -10932.0
xaxis -301 yaxis 38 area 13865.5
xaxis 111 yaxis -189 area -13272.5
xaxis 14 yaxis 201 area 10179.0
xaxis 56 yaxis -132 area -14707.5


In [ ]:
tts=gTTS(text=move,lang='en')
tts.save("ganna.mp3")
os.system("ganna.mp3")